In [1]:
from getpass import getuser # Libaray to copy things
from pathlib import Path # Object oriented libary to deal with paths
import os
from tempfile import NamedTemporaryFile, TemporaryDirectory # Creating temporary Files/Dirs
from subprocess import run, PIPE
import sys

import xarray as xr # Libary to work with labeled n-dimensional data and dask
import numpy as np
import skimage.util as sutil
import matplotlib.pyplot as plt

# sys.path.insert(0, os.path.abspath('/home/mpim/m300414/phd/'))
from org_metrics import Pairs, gen_regionprops_objects_all, gen_shapely_objects_all, gen_tuplelist
from org_metrics import radar_organisation_metric, avg_area, lower_rom_limit

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
data_path = Path('/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/')
glob_pattern_2d = 'bc_notsea*.nc'
 
# Collect all file names with pathlib's rglob and list compressions 
file_names = sorted([str(f) for f in data_path.rglob(f'{glob_pattern_2d}')])
file_names

['/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/bc_notsea_reggrid.nc']

In [4]:
def slide_domain_over_tropics(var_large_field):
    

    domain_size = (117, 117)
    half_size = domain_size[0] // 2
    assert domain_size[0]     == domain_size[1] # domain is quadratic
    assert domain_size[0] % 2 == 1              # number of pixels is not even
    stride_between_domains = half_size + 1
    
    # add half the domain on both right-left sides, but one extra on left side
    halo_field = xr.DataArray(np.zeros(shape=(var_large_field.shape[0],
                                              var_large_field.shape[1] + (2 * half_size) + 1)))
    # Halo swap
    halo_field[:,           :(half_size + 1)] = var_large_field[:, -(half_size + 1):          ]
    halo_field[:, -half_size:               ] = var_large_field[:,                 :half_size ]
    # Inner part
    halo_field[:, (half_size + 1):-half_size] = var_large_field[:, :]
    
    radar_domains = sutil.view_as_windows(
        np.array(halo_field),
        window_shape=domain_size,
        step=stride_between_domains
    )

    map_shape = radar_domains.shape[:2]
    latitude  = var_large_field['lat'][half_size :: stride_between_domains][:map_shape[0]]
    longitude = var_large_field['lon'][0         :: stride_between_domains][:map_shape[1]]
    
    metric_map = xr.DataArray(
        np.zeros(shape=map_shape),
        coords={'lat': latitude, 'lon': longitude},
        dims=('lat', 'lon')
    )
    
    for i in range(map_shape[0]):
        for j in range(map_shape[1]):
            metric_map[i, j] = radar_domains[i, j, :, :].mean()
                    
    return metric_map

In [5]:
var_to_process = xr.open_mfdataset(file_names)['notsea'].squeeze()

In [6]:
# parallelisation on time level
map_singletime = []
map_singletime.append( slide_domain_over_tropics(var_to_process))

In [7]:
result = map_singletime[0]

In [8]:
result

<xarray.DataArray (lat: 29, lon: 272)>
array([[0.00702788, 0.00173144, 0.00055144, ..., 0.12450356, 0.14764502,
        0.02248702],
       [0.08974256, 0.00880875, 0.        , ..., 0.12148735, 0.2114736 ,
        0.10894232],
       [0.08333928, 0.00819809, 0.        , ..., 0.0024916 , 0.07077344,
        0.08838174],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])
Coordinates:
  * lat      (lat) float64 -18.68 -17.36 -16.03 -14.7 ... 14.5 15.83 17.16 18.49
  * lon      (lon) float64 -180.0 -178.7 -177.3 -176.0 ... 177.1 178.4 179.8

In [9]:
result.name = 'land_sea_avg'
result.attrs['units'] = '1'
result.attrs['long_name'] = 'Avg. land-sea mask across (117*2.5)x(117*2.5) km domain.'

In [10]:
result.to_netcdf('/work/mh0731/m300414/DyWinter_b10/RadarDomain_Grid/landsea_experiment.nc')